In [5]:
import pandas as pd 
import pymongo

In [12]:
client = pymongo.MongoClient("localhost", 27017)
db = client["legume-choice"]
print(db.name)
# Extracting the projects data
projectsData =  db["projects"]
# Finding the number of projects
print(projectsData.count_documents({}))

In [30]:
# Extacting all projects
allProjects = []
for project in projectsData.find():
        allProjects.append(project)

# Example querying from the dataset

allProjects[0]["rawdata"]["agroEcoData"]


In [126]:
# Tabular AgroEco scores

def ExtractProjectAgroEcoData(projects,projectIndex):
    projectIndex=0
    projectID =  projects[projectIndex]["projectID"]
    projectName =  projects[projectIndex]["rawdata"]["projectInfo"]["projectName"]

    agroEcoData = projects[projectIndex]["rawdata"]["agroEcoData"]["biofilters"]

    row={"projectID":projectID,"projectName":projectName}


    for index in range(0,len(agroEcoData)):
        row[agroEcoData[index]["label"]]=agroEcoData[index]["value"]
        
    return row

def ExtractAllAgroEcoData(projects):
    agroEcoData=[]
    for projectIndex in range(0,len(projects)):
        agroEcoData.append(ExtractAgroEcoData(projects=projects, projectIndex=projectIndex))
    return pd.DataFrame(agroEcoData)

agroEcoData = ExtractAllAgroEcoData(allProjects)

In [142]:
# Tabular Context Scores


In [190]:
def subsetScoresByVariables(scores, attributeLabel, participantLabel, typologyLabel, average):

    if (average==False):
        attributeLabel=attributes[0]["label"]
        participantLabel=participants[0]["label"]
        typologyLabel=typologies[0]["label"]

        for score in scores:
            if (score["scoreType"]=="individual" and 
            score["attribute"]["label"]==attributeLabel and 
            score["participant"]["label"]==participantLabel and 
            score["typology"]["label"]==typologyLabel):
                 return score["score"]

    if (average==True):

        for score in scores:
            if (score["scoreType"]=="average" and 
            score["attribute"]["label"]==attributeLabel): 
                return score["score"]

    raise ValueError("Did not identify correct subset")
    

def ScoresForAllAttributes(scores, attributes, participant, typology, average):
    listOfScores=[]
    for attribute in attributes:
        listOfScores.append(subsetScoresByVariables(scores, 
        attributeLabel=attribute["label"], 
        participantLabel=participant["label"], 
        typologyLabel=typology["label"], 
        average=average))

    return listOfScores

    
def projectContextScores(projects, projectIndex):

    # Initialising Variables
    projectID =  projects[projectIndex]["projectID"]
    projectName =  projects[projectIndex]["rawdata"]["projectInfo"]["projectName"]


    allScores = allProjects[projectIndex]["rawdata"]["contextScores"]["scores"]
    attributes = allProjects[projectIndex]["rawdata"]["contextScores"]["attributes"]
    participants = allProjects[projectIndex]["rawdata"]["contextScores"]["participants"]
    typologies = allProjects[projectIndex]["rawdata"]["contextScores"]["typologies"]

    # Creating column names for dataframe
    typologyColumns = [ "t_"+typology["label"] for typology in typologies]
    participantColumns = [participant["label"] for participant in participants]
    allColumns = [typ+"_"+particip for typ in typologyColumns for particip in participantColumns]

    # Initialising empty array of scores
    scoresToReturn={}
    column=0
    scoresToReturn["projectName"] = [projectName for attribute in attributes]
    scoresToReturn["projectID"] = [projectID for attribute in attributes]
    for typology in typologies:
        for participant in participants:
            scoresToReturn[allColumns[column]]=ScoresForAllAttributes(scores=allScores, 
                                                                        attributes=attributes,
                                                                        participant=participant, 
                                                                        typology=typology, 
                                                                        average=False )
            column+=1
    
    scoresToReturn["average"]=ScoresForAllAttributes(scores=allScores, attributes=attributes,participant=participants[1], typology=participants[1], average=True )
    return scoresToReturn

In [213]:
pd.DataFrame(scoresToReturn)

,attribute,t_low_farmer,t_low_expert,t_med_farmer,t_med_expert,t_high_farmer,t_high_expert,average
0,land,0,0,0,0,0,0,0
1,labour,0,0,0,0,0,0,0
2,seed,0,0,0,0,0,0,0
3,inp-serv,0,0,0,0,0,0,0
4,knowl-skill,0,0,0,0,0,0,0
5,water,0,0,0,0,0,0,0
6,markets,0,0,0,0,0,0,0


In [219]:
temp=projectContextScores(projects=allProjects, projectIndex=0)
pd.DataFrame(temp)

,projectName,projectID,t_low_farmer,t_low_expert,t_med_farmer,t_med_expert,t_high_farmer,t_high_expert,average
0,faerfsaew,9c93ecbb-51f2-46d6-af77-93b0d54d9428,0,0,0,0,0,0,0
1,faerfsaew,9c93ecbb-51f2-46d6-af77-93b0d54d9428,0,0,0,0,0,0,0
2,faerfsaew,9c93ecbb-51f2-46d6-af77-93b0d54d9428,0,0,0,0,0,0,0
3,faerfsaew,9c93ecbb-51f2-46d6-af77-93b0d54d9428,0,0,0,0,0,0,0
4,faerfsaew,9c93ecbb-51f2-46d6-af77-93b0d54d9428,0,0,0,0,0,0,0
5,faerfsaew,9c93ecbb-51f2-46d6-af77-93b0d54d9428,0,0,0,0,0,0,0
6,faerfsaew,9c93ecbb-51f2-46d6-af77-93b0d54d9428,0,0,0,0,0,0,0


[{'scoreType': 'average',
  'attribute': {'name': 'Land', 'label': 'land'},
  'score': 0},
 {'scoreType': 'individual',
  'attribute': {'name': 'Land', 'label': 'land'},
  'participant': {'name': 'Farmer', 'label': 'farmer'},
  'typology': {'name': 'Low', 'label': 'low'},
  'score': 0},
 {'scoreType': 'individual',
  'attribute': {'name': 'Land', 'label': 'land'},
  'participant': {'name': 'Farmer', 'label': 'farmer'},
  'typology': {'name': 'Medium', 'label': 'med'},
  'score': 0},
 {'scoreType': 'individual',
  'attribute': {'name': 'Land', 'label': 'land'},
  'participant': {'name': 'Farmer', 'label': 'farmer'},
  'typology': {'name': 'High', 'label': 'high'},
  'score': 0},
 {'scoreType': 'individual',
  'attribute': {'name': 'Land', 'label': 'land'},
  'participant': {'name': 'Expert', 'label': 'expert'},
  'typology': {'name': 'Low', 'label': 'low'},
  'score': 0},
 {'scoreType': 'individual',
  'attribute': {'name': 'Land', 'label': 'land'},
  'participant': {'name': 'Expert', '